# Setup and Metrics

In [21]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import mpmath as mp

plt.rcParams['figure.figsize'] = (10.0, 6.0)
plt.rcParams['figure.dpi'] = 120
plt.rcParams['axes.grid'] = True

## Floating-Point Types

In 1985, the Institute of Electrical and Electronics Engineers (IEEE) introduced the **IEEE 754** standard, which defines how computers represent and perform arithmetic on real numbers using a finite number of binary bits.

Most modern programming languages and hardware follow the IEEE 754 standard. For the purpose of this repository, Python and Jupyter notebooks are used, and all floating-point computations are performed using NumPy's IEEE-754 compliant data types: `np.float32` (single precision) and `np.float64` (double precision). These correspond directly to the standard 32-bit and 64-bit IEEE 754 floating-point formats and are used throughout the repository to study precision, rounding behavior, and numerical error.

### Normalized Floating-Point Representation
A (normalized) IEEE 754 floating-point number is represented as:  

$(-1)^{sign} \times 1.fraction \times 2^{exponent}$  

This mirrors scientific notation, but in base 2 instead of base 10.  

#### Components of a Floating-Point Number
**Sign Bit**  
A single bit that determines the sign of the number:  
`0` = Positive  
`1` = Negative  

**Mantissa (Fraction/Significand)**  
Stores the significant digits of the number. For normalized numbers, the leading 1 is implicit.  

**Exponent**  
Stores the scale of the number using a bias so that both positive and negative exponents can be represented.  

### Python (NumPy) Normalized Floating-Point Representation
NumPy's `np.float32` and `np.float64` types follow the IEEE 754 standard for normalized floating-point numbers, differing only in the number of bits allocated to the exponent and mantissa.  

`np.float32` (Single Precision)  
A normalized `np.float32` value is stored using 32 bits:  
Sign: 1 bit  
Exponent: 8 bits (bias = 127)  
Mantissa: 23 bits  
Numerical Representation: $(-1)^{sign} \times 1.fraction \times 2^{exponent-127}$  

`np.float64` (Double Precision)  
A normalized `np.float64` value is stored using 64 bits:  
Sign: 1 bit  
Exponent: 11 bits (bias = 1023)  
Mantissa: 52 bits  
Numerical Representation: $(-1)^{sign} \times 1.fraction \times 2^{exponent-1023}$  

In [22]:
DTYPES = [np.float32, np.float64]

## Error Metrics
The exact real-valued result of computations in floating-point arithmetic is rarely known due to the finite number of bits available in modern computers. Error metrics are used to compare computed **approximations** to a **reference** value that is assumed to be significantly more accurate.  

There are two metrics that will be used: **Absolute Error** and **Relative Error**.  

### Absolute Error
The absolute error is defined as:  

$Absolute\;Error = |Computed\;Value - Reference\;Value|$  

This metric measures the **raw** distance between the computed value and the reference value. Although, simple to implement and interpret the absolute error can be misleading for large magnitude values. An error of $10^{-6}$ may be good or bad depending on the scale of $x$. Because of this, the absolute error is best interpreted alongside relative error.

### Relative Error
The relative error is defined as:  

$Relative\;Error = \frac{Absolute\;Error}{|Reference\;Value|}$  

This metric measures the error **relative** to the magnitude of the reference value and indicates how many correct digits the approximation retains. A drawback of relative error is as the true value becomes close to 0, the metric becomes meaningless. Therefore, the relative error must always be interpreted in context.

In [23]:
def abs_error(true, approx):
    return np.abs(true - approx)

def rel_error(true, approx):
    true = np.asarray(true)
    approx = np.asarray(approx)
    denom = np.maximum(np.abs(true), np.finfo(np.float64).tiny)
    return np.abs(true - approx) / denom

## Reference Value
To quantify floating-point error, a value to compare against is needed. In most real problems the exact real-number result is unknown, so instead a **reference value** is computed and assumed to be significantly more accurate than the floating-point approximation being tested.  

Here reference values are computed using **arbitrary-precision arithmetic** via `mpmath`. This will allow far more precision than `float32` (\~7 decimal digits) and `float64` (\~16 decimal digits), so we can meaningfully measure loss of significance, cancellation, and rounding error effects. 

In [24]:
# Decimal digits of precision
mp.mp.dps = 80

def mp_eval(func, x):
    """
    Evaluate a function at a point x with high precision using mpmath.
    :param func: A function that accepts an mp.mpf and returns an mp.mpf
    :param x: Input value(s)
    :return: np.ndarray or float
    """
    xs = np.atleast_1d(x)
    out = []
    
    for val in xs:
        # Construct mpf from a string to avoid importing binary-float artifacts
        out.append(func(mp.mpf(str(val))))
    
    out = np.array([float(v) for v in out], dtype=np.float64)
    
    return out if np.ndim(x) else out[0]

### Example
Below a high-precision reference for $(1-\cos(x))$ is computed and compared against `float32` and `float64`.

In [25]:
x = 1e-12

ref = mp_eval(lambda t: 1 - mp.cos(t), x)
approx32 = np.float32(1 - np.cos(np.float32(x)))
approx64 = np.float64(1 - np.cos(np.float64(x)))

print("Reference: ", ref)
print("Float32 Approximation: ", approx32)
print("Float64 Approximation: ", approx64)
print()
print("Absolute Errors")
print("Float32: ", abs_error(ref, approx32))
print("Float64: ", abs_error(ref, approx64))
print()
print("Relative Errors")
print("Float32: ", rel_error(ref, approx32))
print("Float64: ", rel_error(ref, approx64))

Reference:  5e-25
Float32 Approximation:  0.0
Float64 Approximation:  0.0

Absolute Errors
Float32:  5e-25
Float64:  5e-25

Relative Errors
Float32:  1.0
Float64:  1.0


## Worked Example: Cancellation in $1 - \cos(x)$

The expression $1 - \cos(x)$ is numerically unstable for small values of $x$. When $x \to 0$, $\cos(x) \approx 1$, and subtracting two nearly equal numbers causes **catastrophic cancellation**, resulting in a severe loss of significant digits in floating-point arithmetic.  

Although the exact value of $1 - \cos(x)$ behaves like $\mathcal{O}(x^2)$ for small $x$, both `float32` and `float64` arithmetic may round $\cos(x)$ to exactly 1.0, causing the computed result to collapse to zero.  

An algebraically equivalent but numerically stable formulation is given by the trigonometric identity:  

$1 - \cos(x) = 2\sin^2\!\left(\frac{x}{2}\right)$  

This reformulation avoids subtracting nearly equal quantities and instead computes small values directly, which is more robust in floating-point arithmetic.

In the following example, we compare the unstable formulation $1 - \cos(x)$ and the stable formulation $2\sin^2(x/2)$ using both `float32` and `float64`, and measure their errors relative to a high-precision reference value computed using arbitrary-precision arithmetic.

In [26]:
x = 1e-12

ref = mp_eval(lambda t: 1 - mp.cos(t), x)

unstable32 = np.float32(1 - np.cos(np.float32(x)))
unstable64 = np.float64(1 - np.cos(np.float64(x)))

stable32 = np.float32(2) * (np.sin(np.float32(x) / np.float32(2)) ** 2)
stable64 = np.float64(2) * (np.sin(np.float64(x) / np.float64(2)) ** 2)

print("Reference:", ref)
print()
print("UNSTABLE: 1 - cos(x)")
print("float32:", unstable32, "abs:", abs_error(ref, unstable32), "rel:", rel_error(ref, unstable32))
print("float64:", unstable64, "abs:", abs_error(ref, unstable64), "rel:", rel_error(ref, unstable64))
print()
print("STABLE: 2*sin(x/2)^2")
print("float32:", stable32, "abs:", abs_error(ref, stable32), "rel:", rel_error(ref, stable32))
print("float64:", stable64, "abs:", abs_error(ref, stable64), "rel:", rel_error(ref, stable64))

Reference: 5e-25

UNSTABLE: 1 - cos(x)
float32: 0.0 abs: 5e-25 rel: 1.0
float64: 0.0 abs: 5e-25 rel: 1.0

STABLE: 2*sin(x/2)^2
float32: 5e-25 abs: 9.770740727281028e-33 rel: 1.9541481454562058e-08
float64: 5e-25 abs: 0.0 rel: 0.0


**Takeaway:** The direct computation $1-\cos(x)$ suffers catastrophic cancellation for small $x$, causing both `float32` and `float64` to collapse to 0. Reformulating the expression using $1-\cos(x)=2\sin^2(x/2)$ avoids subtracting nearly equal numbers and restores accuracy, with `float32` achieving near its precision limit and `float64` matching the reference value in this test.

# References
Wikipedia contributors. (2025, November 29). IEEE 754. In Wikipedia, The Free Encyclopedia. Retrieved 01:07, December 29, 2025, from https://en.wikipedia.org/w/index.php?title=IEEE_754&oldid=1324852683

GeeksforGeeks. (2024, August 5). Absolute error and relative error: Formula and equation. https://www.geeksforgeeks.org/maths/absolute-error-and-relative-error-formula-and-equation/ 

Wikipedia contributors. (2025, February 16). Mean absolute error. In Wikipedia, The Free Encyclopedia. Retrieved 02:05, December 29, 2025, from https://en.wikipedia.org/w/index.php?title=Mean_absolute_error&oldid=1276071917

Wikipedia contributors. (2025, November 18). Approximation error. In Wikipedia, The Free Encyclopedia. Retrieved 02:06, December 29, 2025, from https://en.wikipedia.org/w/index.php?title=Approximation_error&oldid=1322965821